In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                actions = jupyter.actions
                actions.call('jupyter-notebook:run-all-cells-below');
                actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
%matplotlib inline
from os import path as osp
import shutil
from boltons.setutils import IndexedSet
import itertools as itt
import functools as ft
from IPython.display import display, HTML
from ipywidgets import Text, interact, interactive, FlexBox, Box, HTML
import pandas as pd

In [ ]:
import declarativewidgets as declwidgets

declwidgets.init()
HTML('<urth-help/>')

In [ ]:
%%html
<urth-help/>

In [ ]:
%%html
<template is='urth-core-bind'>
    <div>Hello <span>{{user}}</span></div>
    Name: <input value='{{user::input}}'></input>
</template>

In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is="urth-core-import">
<!-- <link rel='import' href='urth_components/paper-button/paper-button.html' 
        is='urth-core-import' debug='1'>
  var body = document.getElementsByTagName("body")[0];
-->
<paper-button raised>Cannot Import Widgets :-(</paper-button>

In [ ]:
from co2mpas.sampling import dice
import gnupg
from os import path as osp

def _describe_gpg(gpg):
    gpg_path = gpg.gpgbinary
    if not osp.isabs(gpg_path):
        gpg_path = shutil.which(gpg_path)
        
    ver = 'version: %s' % str(gpg.version)
    nprivkeys = len(gpg.list_keys(True))
    nallkeys = len(gpg.list_keys())
    return gpg_path, ver, nprivkeys, nallkeys
    
#def collect_gpgs(inc_errors=1, **gpg_kws):
def collect_gpgs():
    print('GG:')
    inc_errors=1
    gpg_kws={}
    gpg_paths = IndexedSet(itt.chain.from_iterable(dice.where(prog) for prog in ('gpg2', 'gpg')))
    gnupghome = osp.expanduser('~/.gnupg')
    gpg_avail = []
    for gpg_path in gpg_paths:
        try:
            gpg = gnupg.GPG(gpgbinary=gpg_path, **gpg_kws)
            row = _describe_gpg(gpg)
        except Exception as ex:
            #raise
            if inc_errors:
                row = (gpg_path, '%s: %s' % (type(ex).__name__, str(ex)), None, None)
            else:
                continue
        gpg_avail.append(row)
    
    cols= ['GnuPG path', 'Description', '# of PRIV keys', '# of PRIV+PUB keys']
    gpg_avail = pd.DataFrame(gpg_avail, columns=cols)
    return gpg_avail


gpg_avail = collect_gpgs()
#display(gpg_avail)

In [ ]:
<!-- 
<urth-core-dataframe ref="gpg_avail" value="{{gpgs_udf}}" auto ></urth-core-dataframe> 
<urth-core-function id='col1' ref="collect_gpgs" result="{{gpgs_udf}}" args="" is-ready="{{ready}}" auto></urth-core-function>
-->

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-dataframe ref="gpg_avail" value="{{gpgs_udf}}" auto ></urth-core-dataframe> 
    <p>LL{{gpgs_udf.data}}KK</p>
</template>

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-dataframe ref="gpg_avail" value="{{gpgs_udf}}" auto ></urth-core-dataframe> 
    <h3>Availabel GnuPG installations: <button onClick="col1.invoke()">Refresh</button></h3>
    <urth-viz-table datarows="[[gpgs_udf.data]]" selection="{{gpg_sel}}" columns="[[gpgs_udf.columns]]" rows-visible=6/>
    <p>DF: {{gpg_udf.data}}, {{ready}}</p>
</template>

In [ ]:
def global_gpg(gpgbinary=""):
    global gpg
    if gpgbinary:
        gpg = gnupg.GPG(gpgbinary=gpgbinary)
    
def collect_priv_keys():
    if gpg:
        return pd.DataFrame(gpg.list_keys(True))

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function ref="global_gpg" arg-gpgbinary="{{gpg_sel.0}}" auto/>
    <urth-core-function ref="collect_priv_keys" result="{{gpg_keys}}" auto/>
    
    <urth-viz-table datarows="[[gpg_keys.data]]" selection="{{gpg_sel}}" columns="[[gpg_keys.columns]]" rows-visible=6/>
</template>

In [ ]:
%%html
<template is='urth-core-bind' >
    <p>{{gpg_sel}}</p>
    Name: <input value='{{df.data}}'></input>
</template>